# Testing Calibration V2Worksheet

## 1. Setup
This section load python objects and variables please check the ```cam``` variable

### 1.1 Load python objects
Loads ```scope``` and ```lamp``` python objects for reading sekonic and communication with the lamp

In [229]:
cam = "http://192.168.2.224/video_feed"
whiteTreshold = 150

%run calibration_setup_v2.ipynb

### 1.2 Connect to lamp
Lamp needs to be closed before it can be opened again.
If you dont know the serial port where the lamp is connected use the Find COM port section
1st You need to setup sekonic and feedback camera (the areas where the data will be read, 1st -green value is set up so it would be possibel to get dUV - sign location), then close the lamp and start again with the scope objects

In [230]:
lamp1.close()

In [199]:
lamp1 = apollo_device(ComPort='COM6')
lamp1.getId()
lamp1.macId = lamp1.macId.replace(":","")
lamp1.macId

'782184456C74'

In [200]:
lamp1.color = [4,4,4,16,0]
lamp1.update()

In [201]:
cmd = "I 1 1 1 255 0\n"
lamp1.send(cmd.encode('utf-8'))
lamp1.read()
print(lamp1.lastValAsJsonString)

{"mixed_red":4,"mixed_green":4,"mixed_blue":4,"mixed_white":16,"red_actual":2,"lamp_temp":24.250000,"lamp_target":65.000000,"fan_speed":0,"Timestamp":1699976598935211200}


#### [Optional] Find COM port

In [9]:
# Get COM port list
import serial.tools.list_ports

com_ports = list(serial.tools.list_ports.comports())

for port, desc, hwid in com_ports:
    print(str(port)+" "+str(desc))

COM5 USB Serial Port (COM5)
COM6 USB Serial Port (COM6)


### 1.3 Set Sekonic camera areas
Set the area where to "read" the Tcp, dUV and lux variables
Once the object is initalized tune the paramters and test by ```update()```
To save the paramteres use the Setup below to generate the ```saved_string``` and copy and paste it into funtion paramter and save the workbook

In [231]:
Brightness = scope(saved_string="self.slider_h.value = (232, 391)\nself.slider_v.value = (373, 431)\nself.minus_comp.value = False\nself.offset_x.value = 0\nself.offset_y.value = 0\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.')]) \n")

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Label(value='Out:')

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x87\x00\x00\x002\x08\x02\x00\x00\x00\xf9\x82_\x9…

Output()

In [12]:
Brightness.update()

4140

In [232]:
dUV = scope(saved_string="self.slider_h.value = (250, 443)\nself.slider_v.value = (289, 353)\nself.minus_comp.value = True\nself.offset_x.value = -80\nself.offset_y.value = 2\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.'), ('20.0', '0,0'), ('0.6', '0.0'), ('0.8', '0.0')]) \n")

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Label(value='Out:')

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x87\x00\x00\x002\x08\x02\x00\x00\x00\xf9\x82_\x9…

Output()

In [14]:
dUV.update()

-0.029

In [233]:
Kelvin = scope(saved_string="self.slider_h.value = (259, 417)\nself.slider_v.value = (215, 273)\nself.minus_comp.value = False\nself.offset_x.value = 0\nself.offset_y.value = 0\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.'), ('G', '6')]) \n")

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Label(value='Out:')

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x87\x00\x00\x002\x08\x02\x00\x00\x00\xf9\x82_\x9…

Output()

In [17]:
Kelvin.update()

5216

#### Setup

In [18]:
Brightness.save_string()

"self.slider_h.value = (232, 391)\nself.slider_v.value = (373, 431)\nself.minus_comp.value = False\nself.offset_x.value = 0\nself.offset_y.value = 0\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.')]) \n"

In [19]:
dUV.save_string()

"self.slider_h.value = (250, 443)\nself.slider_v.value = (289, 353)\nself.minus_comp.value = True\nself.offset_x.value = -80\nself.offset_y.value = 2\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.'), ('20.0', '0,0'), ('0.6', '0.0'), ('0.8', '0.0')]) \n"

In [20]:
Kelvin.save_string()

"self.slider_h.value = (259, 417)\nself.slider_v.value = (215, 273)\nself.minus_comp.value = False\nself.offset_x.value = 0\nself.offset_y.value = 0\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.'), ('G', '6')]) \n"

# Test

In [218]:
lamp1.close()

In [234]:
lamp1 = apollo_device(ComPort='COM6',SekBrightnes=Brightness, SekDuv=dUV, SekKelvin=Kelvin)
lamp1.getId()
lamp1.macId = lamp1.macId.replace(":","")
lamp1.macId

'782184456C74'

In [237]:
%%time

KelvinTemp = 1 # 0-5 [2800 3200 4800 5600 7800 10000]
BrightnessV = 2 # 1-8 [min->max]
readCalibration = True
writeCalibration = True

if readCalibration:
    print("Loading calibartion from lamp\n")
    lamp1.send(("Cr"+str(KelvinTemp)+" "+str(BrightnessV)+" \n").encode('utf-8'))
    lamp1.read()
    jsonString = "{"+lamp1.lastValAsJsonString[1:-1]+"}"
    jsonObject = eval(jsonString)
    
    print("loaded values: "+str(jsonObject)+"\n")
    calibration_points[KelvinTemp][8][2] = jsonObject['r']
    calibration_points[KelvinTemp][8][3] = jsonObject['g']
    calibration_points[KelvinTemp][8][4] = jsonObject['b']
    calibration_points[KelvinTemp][8][5] = jsonObject['w']

    # The answer is 2 line, so to clear the buffer
    lamp1.read()

calibration_points = lamp1.AdjustLux(BrightnessV,KelvinTemp, calibration_points)
calibration_points = lamp1.AdjustDuv(BrightnessV,KelvinTemp, calibration_points)
calibration_points = lamp1.AdjustKelvin(BrightnessV,KelvinTemp, calibration_points)

#write calibration to lamp
if writeCalibration:
    print("Writing calibratrion:\n")
    cmd = "Cw"+str(KelvinTemp)+" "+str(BrightnessV)+" "+str(calibration_points[KelvinTemp][BrightnessV][2])+" "+str(calibration_points[KelvinTemp][BrightnessV][3])+" "+str(calibration_points[KelvinTemp][BrightnessV][4])+" "+str(calibration_points[KelvinTemp][BrightnessV][5])+"\n"
    lamp1.send(cmd.encode('utf-8'))
    print(cmd)
    lamp1.read()
    print(lamp1.lastValAsJsonString)

print(f'Temperature: {lamp1.SekKelvin.update()}K\ndUv: {lamp1.SekDuv.update()}\nBrightness: {lamp1.SekBrightnes.update()}lx')

Loading calibartion from lamp

loaded values: {'wb': 1, 'brightness': 2, 'r': 15, 'g': 9, 'b': 0, 'w': 8, 'Timestamp': 1700034250992717500}

Adjusting lamp brightness to 156lx in 3200K with [2, 1, 0, 1]/[17, 10, 0, 9] steps
current brightness: 153, ideal 156, diff 3 : [15, 8, 0, 10, 0] / [9, 8, 0, 10, 0]
current brightness: 171, ideal 156, diff -15 : [17, 9, 0, 11, 0] / [10, 9, 0, 11, 0]
current brightness: 153, ideal 156, diff 3 : [15, 8, 0, 10, 0] / [9, 8, 0, 10, 0]
current brightness: 171, ideal 156, diff -15 : [17, 9, 0, 11, 0] / [10, 9, 0, 11, 0]
current brightness: 153, ideal 156, diff 3 : [15, 8, 0, 10, 0] / [9, 8, 0, 10, 0]
current brightness: 171, ideal 156, diff -15 : [17, 9, 0, 11, 0] / [10, 9, 0, 11, 0]
current brightness: 153, ideal 156, diff 3 : [15, 8, 0, 10, 0] / [9, 8, 0, 10, 0]
selected: [3200, 2, 15, 8, 0, 10]
Adjusting lamp dUV to +/- 0.003 in 3200K with [0, 1, 0, 0]/[0, 30, 0, 0] steps
current dUV: 0.0062, ideal 0.0000, diff 0.0062 : [15, 8, 0, 10, 0] / [9, 8, 0, 1

In [226]:
%%time

KelvinTemp = 1 # 0-5 [2800 3200 4800 5600 7800 10000]
readCalibration = True
writeCalibration = True

if readCalibration:
    print("Loading calibartion from lamp\n")
    lamp1.send(("Cr"+str(KelvinTemp)+" 8 \n").encode('utf-8'))
    lamp1.read()
    jsonString = "{"+lamp1.lastValAsJsonString[1:-1]+"}"
    jsonObject = eval(jsonString)
    
    print("loaded values: "+str(jsonObject)+"\n")
    calibration_points[KelvinTemp][8][2] = jsonObject['r']
    calibration_points[KelvinTemp][8][3] = jsonObject['g']
    calibration_points[KelvinTemp][8][4] = jsonObject['b']
    calibration_points[KelvinTemp][8][5] = jsonObject['w']

    # The answer is 2 line, so to clear the buffer
    lamp1.read()

for lx in reversed(range(1,9)):
    calibration_points = lamp1.AdjustLux(lx,KelvinTemp, calibration_points)
    calibration_points = lamp1.AdjustDuv(lx,KelvinTemp, calibration_points)
    calibration_points = lamp1.AdjustKelvin(lx,KelvinTemp, calibration_points)
    
    if not(lx==8):
        calibration_points[KelvinTemp][lx] = [round(x/2) for x in calibration_points[KelvinTemp][lx+1]]
    
    #write calibration to lamp
    if writeCalibration:
        print("Writing calibratrion:\n")
        cmd = "Cw"+str(KelvinTemp)+" "+str(lx)+" "+str(calibration_points[KelvinTemp][lx][2])+" "+str(calibration_points[KelvinTemp][lx][3])+" "+str(calibration_points[KelvinTemp][lx][4])+" "+str(calibration_points[KelvinTemp][lx][5])+"\n"
        lamp1.send(cmd.encode('utf-8'))
        print(cmd)
        lamp1.read()
        print(lamp1.lastValAsJsonString)
    
    print(f'Temperature: {lamp1.SekKelvin.update()}K\ndUv: {lamp1.SekDuv.update()}\nBrightness: {lamp1.SekBrightnes.update()}lx')

Loading calibartion from lamp

loaded values: {'wb': 1, 'brightness': 8, 'r': 963, 'g': 594, 'b': 9, 'w': 549, 'Timestamp': 1699990394308052900}

Adjusting lamp brightness to 10000lx in 3200K with [2, 1, 0, 1]/[18, 11, 0, 10] steps
current brightness: 9990, ideal 10000, diff 10 : [963, 594, 9, 549, 0] / [685, 594, 9, 549, 0]
current brightness: 10000, ideal 10000, diff 0 : [965, 595, 9, 550, 0] / [686, 595, 9, 550, 0]
selected: [3200, 8, 965, 595, 9, 550]
Adjusting lamp dUV to +/- 0.003 in 3200K with [0, 1, 0, 0]/[0, 30, 0, 0] steps
current dUV: -0.0001, ideal 0.0000, diff -0.0001 : [965, 595, 9, 550, 0] / [686, 595, 9, 550, 0]
current dUV: 0.0002, ideal 0.0000, diff 0.0002 : [965, 596, 9, 550, 0] / [686, 596, 9, 550, 0]
current dUV: 0.0, ideal 0.0000, diff 0.0 : [965, 595, 9, 550, 0] / [686, 595, 9, 550, 0]
selected: [3200, 8, 965, 595, 9, 550]
Adjusting lamp Kelvin to 3200K with [-1, 0, 0, 1]/[-10, 0, 0, 10] steps
current Kelvin: 3226K, ideal 3200K, diff -26K : [965, 595, 9, 550, 0] 

In [89]:
cmd = "Cr5 8\n"
lamp1.send(cmd.encode('utf-8'))
lamp1.read()
print(lamp1.lastValAsJsonString)

{"wb":5, "brightness":8, "r":417, "g":765, "b":431, "w":1862,"Timestamp":1699952726948475000}


In [197]:
cmd = "Cw5 100 100 30 30 21 0 \n"
lamp1.send(cmd.encode('utf-8'))
lamp1.read()
print(lamp1.lastValAsJsonString)

{,"Timestamp":1699976504400315700}


In [345]:
lamp1.color = [0, 0, 0, 0, 0]
lamp1.update()

In [238]:
lamp1.close()

In [37]:
min_row = min(lamp1.kelvinLast, key=lambda x: abs(x[1]))

In [38]:
min_row

[9257, 743, 7, 8, 6, 31, 0]

In [54]:
cmd = "A 2 5 3 16 0\n"
lamp1.send(cmd.encode('utf-8'))
lamp1.read()
print(lamp1.lastValAsJsonString)

["red_val":1,"uncalibrated_red_val":2,"green_val":5,"blue_val":3,"white_val":16,"fan_val":0,"temp":24.000000,"target_temp_inputC":0.000000,"targetTempDataC":65.000000,"Timestamp":1699975841307912100]


In [49]:
cmd = "I 1 1 1 255 0\n"
lamp1.send(cmd.encode('utf-8'))
lamp1.read()
print(lamp1.lastValAsJsonString)

["red_val":0,"uncalibrated_red_val":1,"green_val":5,"blue_val":3,"white_val":16,"fan_val":384,"temp":24.000000,"target_temp_inputC":20971527.875000,"targetTempDataC":-789473.875000,"Timestamp":1699975575592759900]


In [30]:
cmd = "Cr5 1\n"
lamp1.send(cmd.encode('utf-8'))
lamp1.read()
print(lamp1.lastValAsJsonString)

["wb":5, "brightness":1, "r":2, "g":5, "b":3, "w":14,"Timestamp":1699961041566677300]
